<a href="https://colab.research.google.com/github/abhishek546/music-genre-classification/blob/main/mgc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd "/content/drive/My Drive/samsung"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! nvidia-smi

Mon Jun  8 11:40:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    25W /  75W |   7353MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
 
#!/usr/bin/env python
 
import logging
import os
import tensorflow as tf
import xgboost as xgb
from tensorflow import keras
from keras import layers
from keras import models
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import TimeDistributed
from sklearn.svm import SVC
from keras.utils import to_categorical
 
 
train_X = np.load("/content/drive/My Drive/samsung/rb1/mfe6_train_input1.npy")
train_Y = np.load("/content/drive/My Drive/samsung/rb1/mfe6_train_target1.npy")
dev_X=np.load("/content/drive/My Drive/samsung/rb1/mfe6_validation_input1.npy")
dev_Y=np.load("/content/drive/My Drive/samsung/rb1/mfe6_validation_target1.npy")
test_X=np.load("/content/drive/My Drive/samsung/rb1/mfe6_test_input1.npy")
test_Y=np.load("/content/drive/My Drive/samsung/rb1/mfe6_test_target1.npy")
 
train_X = tf.convert_to_tensor(train_X,dtype = tf.float64)
train_Y = tf.convert_to_tensor(train_Y,dtype = tf.float64)
dev_X = tf.convert_to_tensor(dev_X,dtype = tf.float64)
dev_Y = tf.convert_to_tensor(dev_Y,dtype = tf.float64)
test_X = tf.convert_to_tensor(test_X,dtype = tf.float64)
test_Y = tf.convert_to_tensor(test_Y,dtype = tf.float64)
print(train_X.shape)
input_shape = (train_X.shape[1], train_X.shape[2],train_X.shape[3])
print("Build LSTM RNN model ...")
model = Sequential()
 
model = keras.Sequential()
model.add(keras.layers.Conv2D(128,(3,3),activation='relu',input_shape=input_shape,padding='same'))
model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding='same'))
model.add(keras.layers.BatchNormalization())
 
model.add(keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape,padding='same'))
model.add(keras.layers.MaxPool2D((3,3),strides=(2,2),padding='same'))
model.add(keras.layers.BatchNormalization())
 
model.add(keras.layers.Conv2D(64,(3,3),activation='relu',input_shape=input_shape,padding='same'))
model.add(keras.layers.MaxPool2D((2,4),strides=(2,2),padding='same'))
model.add(keras.layers.BatchNormalization())
 
model.add(keras.layers.Conv2D(32,(2,2),activation='relu',input_shape=input_shape,padding='same'))
model.add(keras.layers.MaxPool2D((2,4),strides=(2,2),padding='same'))
model.add(keras.layers.BatchNormalization())
 
model.add(keras.layers.Flatten())
#model.add(keras.layers.Dense(2048, activation= 'relu'))
#model.add(keras.layers.Dropout(0.8))
model.add(keras.layers.Dense(1024, activation= 'relu'))
model.add(keras.layers.Dropout(0.7))
model.add(keras.layers.Dense(512, activation= 'relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(256, activation= 'relu'))
model.add(keras.layers.Dropout(0.35))
model.add(keras.layers.Dense(128, activation= 'relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
#model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
optimerz = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimerz,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
model.fit(train_X,train_Y,
          validation_data=(dev_X,dev_Y), epochs= 100)
test_accuracy= model.evaluate(test_X,test_Y,verbose=1)
print("Accuracy: ".format(test_accuracy))
print(test_accuracy)
print("Training ...")
print(model.summary())
from pathlib import Path
import pandas as pd
 
 
# Creates a HDF5 file 'lstm_genre_classifier.h5'
model_filename = "/content/drive/My Drive/samsung/rb1/ch.hdf5"
 
print("\nSaving model: " + model_filename)
model.save(model_filename)
print("creating .json file....")
model_json = model.to_json()
f = Path("/content/drive/My Drive/samsung/rb1/ch.json")
f.write_text(model_json)
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
"An example of predicting a music genre from a custom audio file"
import librosa
import logging
import sys
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras.optimizers import Adam
 # local python class with Audio feature extraction and genre list
 
# set logging level
logging.getLogger("tensorflow").setLevel(logging.ERROR)
genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock"
    ]
 
def load_model(model_path, weights_path):
    "Load the trained LSTM model from directory for genre classification"
    with open(model_path, "r") as model_file:
        trained_model = tf.keras.models.model_from_json(model_file.read())
    trained_model.load_weights(weights_path)
    optimerz = keras.optimizers.Adam(learning_rate=0.0001)
    trained_model.compile(
        optimizer=optimerz,
                loss='categorical_crossentropy',
                metrics=['accuracy']
    )
    return trained_model
 
 
def extract_audio_features1(file,j):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros(
        (1,timeseries_length,128,1), dtype=np.float64
        )
 
    y, sr = librosa.load(file,offset=j*3,duration=6)
    
    spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
    spect = librosa.power_to_db(spect, ref=np.max)
    if spect.shape[1] != 128:
         spect.resize(128,128, refcheck=False)
    features[0,:,0:128,0] = spect
    return features
def extract_audio_features2(file,j):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros(
        (1,timeseries_length,512,1), dtype=np.float64
        )
 
    y, sr = librosa.load(file,offset=j*3,duration=6)
    
    X = librosa.stft(y)
    X = librosa.amplitude_to_db(abs(X))
    if X.shape[1] != 512:
         X.resize(128,512, refcheck=False)
    features[0,:,0:512,0] = X
    return features
def extract_audio_features3(file,j):
    "Extract audio features from an audio file for genre classification"
    timeseries_length = 128
    features = np.zeros(
        (1,timeseries_length,20,1), dtype=np.float64
        )
 
    y, sr = librosa.load(file,offset=j*3,duration=6)
    mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=1024, n_mfcc=20
            )
    spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=1024
            )
    chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=1024)
    spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=1024
            )
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    features[0, :, 0:20,0] = mfcc.T[0:128,:]
    
    return features
 
def get_genre(model1,model2,model3,music_path):
    "Predict genre of music using a trained model"
    A=[]
    for j in range(9):
      prediction = model1.predict(extract_audio_features1(music_path,j))
      predict_genre = genre_list[np.argmax(prediction)]
      A.append(predict_genre)
      #print(predict_genre)
    for i in range(9):
      prediction = model2.predict(extract_audio_features2(music_path,i))
      predict_genre = genre_list[np.argmax(prediction)]
      #A.append(predict_genre)
      #print(predict_genre)
    
    #for k in range(9):
     # prediction = model3.predict(extract_audio_features3(music_path,k))
     # predict_genre = genre_list[np.argmax(prediction)] 
      
      
      #A.append(predict_genre)
      #print(predict_genre)
    new_list = sorted(A, key = A.count, reverse=True)
    return new_list[0]
 
 
c = 0
co = 0
A = []
B = []
history = (A,B)
 
dict_1 = history
 
if __name__ == "__main__":
    import pathlib
    import os
    path = "/content/drive/My Drive/samsung/test1/"
    data_dir = pathlib.Path(path)
    K = list(data_dir.glob("*"))
    MODEL1 = load_model("/content/drive/My Drive/samsung/rb1/dm6.json", "/content/drive/My Drive/samsung/rb1/dm6.hdf5")
    MODEL2 = load_model("/content/drive/My Drive/samsung/6d/ds6.json", "/content/drive/My Drive/samsung/6d/ds6.hdf5")
    MODEL3 = load_model("/content/drive/My Drive/samsung/6d/mf6.json", "/content/drive/My Drive/samsung/6d/mf6.hdf5")
    for i in range(len(K)):
        co=co+1
        p1 = str(K[i])
        GENRE = get_genre(MODEL1,MODEL2,MODEL3,p1)
        k = str(K[i])
        m1 = os.path.splitext(k)
        m2 = os.path.split(m1[0])
        B.append("Model predict: {}".format(m2[1][:-6]))
        A.append("Model predict: {}".format(GENRE))
        if A[i]==B[i]:
            c=c+1
        print("Model predict: "+GENRE+" Actual model is "+m2[1][:-6])
 
import pandas as pd
df1 = pd.DataFrame.from_dict(dict_1)
print("Accuracy of the model")
print(c/co)

Model predict: blues Actual model is blues
Model predict: jazz Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: rock Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: disco Actual model is blues
Model predict: jazz Actual model is blues
Model predict: blues Actual model is blues
Model predict: jazz Actual model is blues
Model predict: reggae Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: rock Actual model is blues
Model predict: jazz Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: blues Actual model is blues
Model predict: jazz Actual model is blues
Model predict: classical Actual model is classical
Model pre

In [ ]:
import librosa
import math
import os
import re

import numpy as np


class wor:

    "Music audio features for genre classification"
    hop_length = None
    genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock"
    ]

    dir_trainfolder = "/content/drive/My Drive/samsung/train"
    dir_devfolder = "/content/drive/My Drive/samsung/validation"
    dir_testfolder = "/content/drive/My Drive/samsung/test"
    dir_all_files = "/content/drive/My Drive/samsung"

    train_X_preprocessed_data = "/content/drive/My Drive/samsung/mf6_train_input1.npy"
    train_Y_preprocessed_data = "/content/drive/My Drive/samsung/mf6_train_target1.npy"
    dev_X_preprocessed_data = "/content/drive/My Drive/samsung/mf6_validation_input1.npy"
    dev_Y_preprocessed_data = "/content/drive/My Drive/samsung/mf6_validation_target1.npy"
    test_X_preprocessed_data = "/content/drive/My Drive/samsung/mf6_test_input1.npy"
    test_Y_preprocessed_data = "/content/drive/My Drive/samsung/mf6_test_target1.npy"
    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 1024

        self.timeseries_length_list = []
        self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
        self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
        self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        self.all_files_list = []
        self.all_files_list.extend(self.trainfiles_list)
        self.all_files_list.extend(self.devfiles_list)
        self.all_files_list.extend(self.testfiles_list)

        # compute minimum timeseries length, slow to compute, caching pre-computed value of 1290
        # self.precompute_min_timeseries_len()
        # print("min(self.timeseries_length_list) ==" + str(min(self.timeseries_length_list)))
        # self.timeseries_length = min(self.timeseries_length_list)

        self.timeseries_length = (
            128
        )   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
        #  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

    def load_preprocess_data(self):
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X, self.train_Y = self.extract_audio_features(self.trainfiles_list)
        with open(self.train_X_preprocessed_data, "wb") as f:
            np.save(f, self.train_X)
        with open(self.train_Y_preprocessed_data, "wb") as f:
            self.train_Y = self.one_hot(self.train_Y)
            np.save(f, self.train_Y)

        # Validation set
        self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
        with open(self.dev_X_preprocessed_data, "wb") as f:
            np.save(f, self.dev_X)
        with open(self.dev_Y_preprocessed_data, "wb") as f:
            self.dev_Y = self.one_hot(self.dev_Y)
            np.save(f, self.dev_Y)

        # Test set
        self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
        with open(self.test_X_preprocessed_data, "wb") as f:
            np.save(f, self.test_X)
        with open(self.test_Y_preprocessed_data, "wb") as f:
            self.test_Y = self.one_hot(self.test_Y)
            np.save(f, self.test_Y)

    def load_deserialize_data(self):

        self.train_X = np.load(self.train_X_preprocessed_data)
        self.train_Y = np.load(self.train_Y_preprocessed_data)

        self.dev_X = np.load(self.dev_X_preprocessed_data)
        self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
        self.test_Y = np.load(self.test_Y_preprocessed_data)
    

    def precompute_min_timeseries_len(self):
        for file in self.all_files_list:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):

        features = np.zeros(
        (len(list_of_audiofiles)*9,self.timeseries_length,40,1), dtype=np.float64
        )
        target = []
        c=0
        for i, file in enumerate(list_of_audiofiles):
          for j in range(9):
            y, sr = librosa.load(file,offset= j*3 ,duration=6)
            mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=self.hop_length, n_mfcc=20
            )
            spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=self.hop_length
            )
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
            spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=self.hop_length
            )
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            X = librosa.stft(y)
            X = librosa.amplitude_to_db(abs(X))
            spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
            spect = librosa.power_to_db(spect, ref=np.max)
            if spect.shape[1] != 128:
                spect.resize(128,128, refcheck=False)
            if X.shape[1] != 128:
               X.resize((128,128), refcheck=False)
            splits = re.split("[ .]", file)
            genre = re.split("[ /]", splits[1])[3]
            target.append(genre)
            print(genre)
            k=0
            for e in mfcc:
             features[c, :, k,0]= np.mean(e)
             k=k+1
             features[c, :, k,0]= np.std(e)
             k=k+1
            
            c=c+1
            print(
                "Extracted features audio track %i of %i."
                % (c + 1, len(list_of_audiofiles)*9)
            )
        return features, np.expand_dims(np.asarray(target), axis=1)

    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0],10))
        for i, genre_string in enumerate(Y_genre_strings):
            index = self.genre_list.index(genre_string)
            y_one_hot[i,index] = 1 
        return y_one_hot

    @staticmethod
    def path_to_audiofiles(dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".au"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio
        
gen=wor()
gen.load_preprocess_data()

In [ ]:
import librosa
import math
import os
import re

import numpy as np


class wor:

    "Music audio features for genre classification"
    hop_length = None
    genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock"
    ]

    dir_trainfolder = "/content/drive/My Drive/samsung/train"
    dir_devfolder = "/content/drive/My Drive/samsung/validation"
    dir_testfolder = "/content/drive/My Drive/samsung/test"
    dir_all_files = "/content/drive/My Drive/samsung"

    train_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_train_input1.npy"
    train_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_train_target1.npy"
    dev_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_validation_input1.npy"
    dev_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_validation_target1.npy"
    test_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_test_input1.npy"
    test_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dse6_test_target1.npy"
    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 1024

        self.timeseries_length_list = []
        self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
        self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
        self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        self.all_files_list = []
        self.all_files_list.extend(self.trainfiles_list)
        self.all_files_list.extend(self.devfiles_list)
        self.all_files_list.extend(self.testfiles_list)

        # compute minimum timeseries length, slow to compute, caching pre-computed value of 1290
        # self.precompute_min_timeseries_len()
        # print("min(self.timeseries_length_list) ==" + str(min(self.timeseries_length_list)))
        # self.timeseries_length = min(self.timeseries_length_list)

        self.timeseries_length = (
            128
        )   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
        #  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

    def load_preprocess_data(self):
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X, self.train_Y = self.extract_audio_features(self.trainfiles_list)
        with open(self.train_X_preprocessed_data, "wb") as f:
            np.save(f, self.train_X)
        with open(self.train_Y_preprocessed_data, "wb") as f:
            self.train_Y = self.one_hot(self.train_Y)
            np.save(f, self.train_Y)

        # Validation set
        self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
        with open(self.dev_X_preprocessed_data, "wb") as f:
            np.save(f, self.dev_X)
        with open(self.dev_Y_preprocessed_data, "wb") as f:
            self.dev_Y = self.one_hot(self.dev_Y)
            np.save(f, self.dev_Y)

        # Test set
        self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
        with open(self.test_X_preprocessed_data, "wb") as f:
            np.save(f, self.test_X)
        with open(self.test_Y_preprocessed_data, "wb") as f:
            self.test_Y = self.one_hot(self.test_Y)
            np.save(f, self.test_Y)

    def load_deserialize_data(self):
        
        self.train_X = np.load(self.train_X_preprocessed_data)
        self.train_Y = np.load(self.train_Y_preprocessed_data)
        
        self.dev_X = np.load(self.dev_X_preprocessed_data)
        self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
        self.test_Y = np.load(self.test_Y_preprocessed_data)
    

    def precompute_min_timeseries_len(self):
        for file in self.all_files_list:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):

        features = np.zeros(
        (len(list_of_audiofiles)*9,self.timeseries_length,128,1), dtype=np.float64
        )
        target = []
        c=0
        for i, file in enumerate(list_of_audiofiles):
          for j in range(9):
            y, sr = librosa.load(file,offset= j*3 ,duration=6)
            mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=self.hop_length, n_mfcc=20
            )
            spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=self.hop_length
            )
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
            spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=self.hop_length
            )
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            X = librosa.stft(y)
            X = librosa.amplitude_to_db(abs(X))
            spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
            spect = librosa.power_to_db(spect, ref=np.max)
            if spect.shape[1] != 128:
                spect.resize(128,128, refcheck=False)
            if X.shape[1] != 128:
               X.resize((128,128), refcheck=False)
            splits = re.split("[ .]", file)
            genre = re.split("[ /]", splits[1])[3]
            target.append(genre)
            print(genre)
            features[c, :, 0:128,0]= X
            
            
            c=c+1
            print(
                "Extracted features audio track %i of %i."
                % (c + 1, len(list_of_audiofiles)*10)
            )
        return features, np.expand_dims(np.asarray(target), axis=1)

    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0],10))
        for i, genre_string in enumerate(Y_genre_strings):
            index = self.genre_list.index(genre_string)
            y_one_hot[i,index] = 1 
        return y_one_hot

    @staticmethod
    def path_to_audiofiles(dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".au"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio
class wor1:

    "Music audio features for genre classification"
    hop_length = None
    genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock"
    ]

    dir_trainfolder = "/content/drive/My Drive/samsung/train"
    dir_devfolder = "/content/drive/My Drive/samsung/validation"
    dir_testfolder = "/content/drive/My Drive/samsung/test"
    dir_all_files = "/content/drive/My Drive/samsung"

    train_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_train_input1.npy"
    train_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_train_target1.npy"
    dev_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_validation_input1.npy"
    dev_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_validation_target1.npy"
    test_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_test_input1.npy"
    test_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/dm6_test_target1.npy"
    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 1024

        self.timeseries_length_list = []
        self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
        self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
        self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        self.all_files_list = []
        self.all_files_list.extend(self.trainfiles_list)
        self.all_files_list.extend(self.devfiles_list)
        self.all_files_list.extend(self.testfiles_list)

        # compute minimum timeseries length, slow to compute, caching pre-computed value of 1290
        # self.precompute_min_timeseries_len()
        # print("min(self.timeseries_length_list) ==" + str(min(self.timeseries_length_list)))
        # self.timeseries_length = min(self.timeseries_length_list)

        self.timeseries_length = (
            128
        )   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
        #  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

    def load_preprocess_data(self):
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X, self.train_Y = self.extract_audio_features(self.trainfiles_list)
        with open(self.train_X_preprocessed_data, "wb") as f:
            np.save(f, self.train_X)
        with open(self.train_Y_preprocessed_data, "wb") as f:
            self.train_Y = self.one_hot(self.train_Y)
            np.save(f, self.train_Y)

        # Validation set
        self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
        with open(self.dev_X_preprocessed_data, "wb") as f:
            np.save(f, self.dev_X)
        with open(self.dev_Y_preprocessed_data, "wb") as f:
            self.dev_Y = self.one_hot(self.dev_Y)
            np.save(f, self.dev_Y)

        # Test set
        self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
        with open(self.test_X_preprocessed_data, "wb") as f:
            np.save(f, self.test_X)
        with open(self.test_Y_preprocessed_data, "wb") as f:
            self.test_Y = self.one_hot(self.test_Y)
            np.save(f, self.test_Y)

    def load_deserialize_data(self):

        self.train_X = np.load(self.train_X_preprocessed_data)
        self.train_Y = np.load(self.train_Y_preprocessed_data)

        self.dev_X = np.load(self.dev_X_preprocessed_data)
        self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
        self.test_Y = np.load(self.test_Y_preprocessed_data)
    

    def precompute_min_timeseries_len(self):
        for file in self.all_files_list:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):

        features = np.zeros(
        (len(list_of_audiofiles)*9,self.timeseries_length,128,1), dtype=np.float64
        )
        target = []
        c=0
        for i, file in enumerate(list_of_audiofiles):
          for j in range(9):
            y, sr = librosa.load(file,offset= j*3 ,duration=6)
            mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=self.hop_length, n_mfcc=20
            )
            spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=self.hop_length
            )
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
            spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=self.hop_length
            )
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            X = librosa.stft(y)
            X = librosa.amplitude_to_db(abs(X))
            spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
            spect = librosa.power_to_db(spect, ref=np.max)
            if spect.shape[1] != 128:
                spect.resize(128,128, refcheck=False)
            if X.shape[1] != 512:
               X.resize((128,512), refcheck=False)
            splits = re.split("[ .]", file)
            genre = re.split("[ /]", splits[1])[3]
            target.append(genre)
            print(genre)
            features[c, :, 0:128,0]= spect
            
            c=c+1
            print(
                "Extracted features audio track %i of %i."
                % (c + 1, len(list_of_audiofiles)*10)
            )
        return features, np.expand_dims(np.asarray(target), axis=1)

    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0],10))
        for i, genre_string in enumerate(Y_genre_strings):
            index = self.genre_list.index(genre_string)
            y_one_hot[i,index] = 1 
        return y_one_hot

    @staticmethod
    def path_to_audiofiles(dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".au"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio
class wor2:

    "Music audio features for genre classification"
    hop_length = None
    genre_list = [
        "blues",
        "classical",
        "country",
        "disco",
        "hiphop",
        "jazz",
        "metal",
        "pop",
        "reggae",
        "rock"
    ]

    dir_trainfolder = "/content/drive/My Drive/samsung/train"
    dir_devfolder = "/content/drive/My Drive/samsung/validation"
    dir_testfolder = "/content/drive/My Drive/samsung/test"
    dir_all_files = "/content/drive/My Drive/samsung"

    train_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_train_input1.npy"
    train_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_train_target1.npy"
    dev_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_validation_input1.npy"
    dev_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_validation_target1.npy"
    test_X_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_test_input1.npy"
    test_Y_preprocessed_data = "/content/drive/My Drive/samsung/rb1/mfe6_test_target1.npy"
    train_X = train_Y = None
    dev_X = dev_Y = None
    test_X = test_Y = None

    def __init__(self):
        self.hop_length = 1024

        self.timeseries_length_list = []
        self.trainfiles_list = self.path_to_audiofiles(self.dir_trainfolder)
        self.devfiles_list = self.path_to_audiofiles(self.dir_devfolder)
        self.testfiles_list = self.path_to_audiofiles(self.dir_testfolder)

        self.all_files_list = []
        self.all_files_list.extend(self.trainfiles_list)
        self.all_files_list.extend(self.devfiles_list)
        self.all_files_list.extend(self.testfiles_list)

        # compute minimum timeseries length, slow to compute, caching pre-computed value of 1290
        # self.precompute_min_timeseries_len()
        # print("min(self.timeseries_length_list) ==" + str(min(self.timeseries_length_list)))
        # self.timeseries_length = min(self.timeseries_length_list)

        self.timeseries_length = (
            128
        )   # sequence length == 128, default fftsize == 2048 & hop == 512 @ SR of 22050
        #  equals 128 overlapped windows that cover approx ~3.065 seconds of audio, which is a bit small!

    def load_preprocess_data(self):
        print("[DEBUG] total number of files: " + str(len(self.timeseries_length_list)))

        # Training set
        self.train_X, self.train_Y = self.extract_audio_features(self.trainfiles_list)
        with open(self.train_X_preprocessed_data, "wb") as f:
            np.save(f, self.train_X)
        with open(self.train_Y_preprocessed_data, "wb") as f:
            self.train_Y = self.one_hot(self.train_Y)
            np.save(f, self.train_Y)

        # Validation set
        self.dev_X, self.dev_Y = self.extract_audio_features(self.devfiles_list)
        with open(self.dev_X_preprocessed_data, "wb") as f:
            np.save(f, self.dev_X)
        with open(self.dev_Y_preprocessed_data, "wb") as f:
            self.dev_Y = self.one_hot(self.dev_Y)
            np.save(f, self.dev_Y)

        # Test set
        self.test_X, self.test_Y = self.extract_audio_features(self.testfiles_list)
        with open(self.test_X_preprocessed_data, "wb") as f:
            np.save(f, self.test_X)
        with open(self.test_Y_preprocessed_data, "wb") as f:
            self.test_Y = self.one_hot(self.test_Y)
            np.save(f, self.test_Y)

    def load_deserialize_data(self):

        self.train_X = np.load(self.train_X_preprocessed_data)
        self.train_Y = np.load(self.train_Y_preprocessed_data)

        self.dev_X = np.load(self.dev_X_preprocessed_data)
        self.dev_Y = np.load(self.dev_Y_preprocessed_data)

        self.test_X = np.load(self.test_X_preprocessed_data)
        self.test_Y = np.load(self.test_Y_preprocessed_data)
    

    def precompute_min_timeseries_len(self):
        for file in self.all_files_list:
            print("Loading " + str(file))
            y, sr = librosa.load(file)
            self.timeseries_length_list.append(math.ceil(len(y) / self.hop_length))

    def extract_audio_features(self, list_of_audiofiles):

        features = np.zeros(
        (len(list_of_audiofiles)*9,128,13,1), dtype=np.float64
        )
        target = []
        c=0
        for i, file in enumerate(list_of_audiofiles):
          for j in range(9):
            y, sr = librosa.load(file,offset= j*3 ,duration=6)
            mfcc = librosa.feature.mfcc(
                y=y, sr=sr, hop_length=self.hop_length, n_mfcc=13
            )
            spectral_center = librosa.feature.spectral_centroid(
                y=y, sr=sr, hop_length=self.hop_length
            )
            chroma = librosa.feature.chroma_stft(y=y, sr=sr, hop_length=self.hop_length)
            spectral_contrast = librosa.feature.spectral_contrast(
                y=y, sr=sr, hop_length=self.hop_length
            )
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = librosa.feature.zero_crossing_rate(y)
            X = librosa.stft(y)
            X = librosa.amplitude_to_db(abs(X))
            spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=1024)
            spect = librosa.power_to_db(spect, ref=np.max)
            if spect.shape[1] != 128:
                spect.resize(128,128, refcheck=False)
            if X.shape[1] != 512:
               X.resize((128,512), refcheck=False)
            
            splits = re.split("[ .]", file)
            genre = re.split("[ /]", splits[1])[3]
            target.append(genre)
            print(genre)
            features[c,:,0:13,0]=mfcc.T[0:128,:]
            
            c=c+1
            print(
                "Extracted features audio track %i of %i."
                % (c + 1, len(list_of_audiofiles)*10)
            )
        return features, np.expand_dims(np.asarray(target), axis=1)

    def one_hot(self, Y_genre_strings):
        y_one_hot = np.zeros((Y_genre_strings.shape[0],10))
        for i, genre_string in enumerate(Y_genre_strings):
            index = self.genre_list.index(genre_string)
            y_one_hot[i,index] = 1 
        return y_one_hot

    @staticmethod
    def path_to_audiofiles(dir_folder):
        list_of_audio = []
        for file in os.listdir(dir_folder):
            if file.endswith(".au"):
                directory = "%s/%s" % (dir_folder, file)
                list_of_audio.append(directory)
        return list_of_audio

gen=wor()
gen1=wor1()
gen2=wor2()
gen2.load_preprocess_data()
#gen1.load_preprocess_data()
gen.load_preprocess_data()